In [1]:
import numpy as np
import os.path as osp

from general_tools.in_out.basics import create_dir
from tf_lab.evaluate.generative_pc_nets import entropy_of_occupancy_grid, jensen_shannon_divergence
from tf_lab.nips.helper import center_pclouds_in_unit_sphere, pclouds_centered_and_half_sphere, pclouds_with_zero_mean_in_unit_sphere

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
cmp_in_sphere = True
voxel_resolution = 28
class_name = 'chair'
save_res = False
ae_loss = 'emd'
def identity(x):
    return x

# pc_normalizer = pclouds_with_zero_mean_in_unit_sphere
pc_normalizer = identity

In [4]:
# Load Ground-Truth Train Data.
top_gt_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/gt_data/'
gt_train_file = osp.join(top_gt_dir, class_name + '_train.npz')
gt_train_data = np.load(gt_train_file)  
gt_train_data = pc_normalizer(gt_train_data[gt_train_data.keys()[0]])
_, gt_train_var = entropy_of_occupancy_grid(gt_train_data, voxel_resolution, in_sphere=cmp_in_sphere)    

# # Load Ground-Truth Test Data.
# gt_test_file = osp.join(top_gt_dir, class_name + '_test.npz')
# gt_test_data = np.load(gt_test_file)  
# gt_test_data = pc_normalizer(gt_test_data[gt_test_data.keys()[0]])
# _, gt_test_var = entropy_of_occupancy_grid(gt_test_data, voxel_resolution, in_sphere=cmp_in_sphere)

In [8]:
# Specify where to load synthetic data.
top_in_dir = osp.join('/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/gmm', ae_loss)
top_synthetic_dir = osp.join(top_in_dir, 'bneck_128_full_20_gaussians')

In [11]:
if save_res:
    out_file = '_'.join([special_tag, pc_normalizer.__name__])
    out_file = osp.join(top_out_dir, out_file + '.txt')
    fout = open(out_file, 'w', 1)
    fout.write('#Metric Epoch (Train-Test) Measurements\n')
    print 'Saving measurements at: ' + out_file
    
sample_file = top_synthetic_dir + '.npz'
sample_data = np.load(sample_file)
sample_data = sample_data[sample_data.keys()[0]]
sample_data = pc_normalizer(sample_data)
_, sample_var = entropy_of_occupancy_grid(sample_data, voxel_resolution, in_sphere=cmp_in_sphere)        

jsd_train = jensen_shannon_divergence(gt_train_var, sample_var)
jsd_test = jsd_train
#     jensen_shannon_divergence(gt_test_var, sample_var)

log_data = 'JSD %f %f' % (jsd_train, jsd_test)
print log_data
if save_res:
    fout.write(log_data + '\n')
if save_res:
    fout.close()

JSD 0.006170 0.006170


(5761, 2048, 3)

In [16]:
from tf_lab.evaluate.generative_pc_nets import minimum_mathing_distance
reduce_samples = 1000
reduce_gt = 1000
batch_size = 1000
if reduce_samples is not None:
    sample_data = sample_data[:reduce_samples]
if reduce_gt is not None:
    gt_train_data = gt_train_data[:reduce_gt]

mmd_train = minimum_mathing_distance(sample_data, gt_train_data, batch_size, normalize=True, use_sqrt=True, verbose=True)[0]
print mmd_train

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
0.0577437
